In [2]:
"""
Purpose: To provide a hands on demonstration of the 
volume_data_interfaces and the parameter configs that 
work together to configure NEURD for any dataset
"""
""

''

# The problem

In [3]:
"""
1) Originally developed code for MICrONS dataset
- organized code as functions collected in python modules
    -> used global variables inside modules for sharing
2) Used datajoint for storing of all intermediate products
and final data products

 
Problems:
1) Expand to other datasets (h01)
2) Make package extensible for non-datajoint users
    a. Input meshes/synapses and volume configurations coming from datajoint
    b. Intermediate/final dataproducts going to datajoint
3) little time to refactor

"""
_

''

# Solution 1: Parameter Changes

In [4]:
"""
Problem: How to adjust parameters for different volumes

Solution 1: How to change/set all the parameters
1) Wrote neurd/parameter_utils.py
    Input: mulitple parameter_config.py files
    Output: for every module referenced, 
            sets the attributes/global parameters referenced
            according to the configuration files

------ Example 1: ------

neurd/parameter_configs/parameters_config_default.py
parameters = {
'axon_utils':
  {
    'global_parameters':
    {
      'auto_proof':
      {
        'ray_trace_min_axon_spines':270,
        'ray_trace_max_axon_spines':1200,
        ...
      },
      'axon_finding':
      {
        'axon_soma_angle_threshold_excitatory':70,
        'ais_max_distance_from_soma_excitatory':14000,
    },
    'attributes':
    {
      'no_category':
      {
        'max_ais_distance_from_soma':50000,
      },
    },
  },
}
- There are parameters in NEURD/axon_utils.py called
        -- global  (with "global" suffix)
            'ray_trace_min_axon_spines_global'
            'axon_soma_angle_threshold_excitatory_global' 
        -- attributes: (without "global" suffix)
            'max_ais_distance_from_soma'

Process:
1) Define a set of parameter_config[].py files
    - usually only define one that overwrites defaults
2) Create an object package_utils.PackageParameters with
    all the parameter information
3) Initialize all the parameters in the modules in the directory
"""
_

''

In [9]:
from neurd.parameter_configs import (
parameters_config_default as par_default,
parameters_config_h01 as par_h01
)

In [11]:
par_default.parameters

{'apical_utils': {'global_parameters': {'apical': {'soma_angle_to_apical': 60,
    'multi_apical_height': -460000,
    'max_upward_angle_shaft_like': 30,
    'min_upward_length_shaft_like': 3000,
    'min_upward_per_match_shaft_like': 0.8,
    'min_upward_length_backup_shaft_like': 20000,
    'min_upward_per_match_backup_shaft_like': 0.5,
    'width_min_shaft_like': 140,
    'min_skeletal_length_filter_apical': 10000,
    'min_distance_above_soma_filter_apical': 10000,
    'candidate_connected_component_radius_apical': 5000,
    'multi_apical_possible_apical': True,
    'width_min_apical_high_soma': 450,
    'distance_from_soma_apical_high_soma': 80000,
    'min_thick_near_soma_skeletal_length_apical_high_soma': 10000,
    'non_upward_skeletal_distance_upstream_buffer_filter_apical_one': -10000,
    'soma_diff_buffer_filter_apical_one': -50000,
    'downstream_vector_diff_buffer_filter_apical_one': -30000,
    'default_tie_breaker_filter_apical_one': 'skeletal_length',
    'add_low_deg

In [10]:
par_h01.parameters

{'apical_utils': {'global_parameters': {'apical': {'multi_apical_possible_apical': False}},
  'attributes': {'no_category': {}}},
 'axon_utils': {'global_parameters': {'auto_proof': {'skeletal_length_max_axon_spines': 12000},
   'axon_finding': {'axon_classification_without_synapses_excitatory': True,
    'ais_width_max': 900,
    'ais_new_width_min': 140,
    'min_skeletal_length': 10000,
    'ais_syn_density_max_excitatory': 100000,
    'ais_syn_density_max_backup_excitatory': 100000,
    'ais_new_width_min_inhibitory': 140,
    'n_postsyn_max': 100000,
    'postsyn_distance': 90000,
    'max_search_distance_addition_backup': 10000},
   'axon_on_dendrite': {'n_synapses_pre_min_ax_on_dendr': 0,
    'synapse_pre_perc_min_ax_on_dendr': 0,
    'width_max_ax_on_dendr': 350,
    'dendrite_width_min_ax_on_dendr': 200},
   'bouton_webbing': {'max_bouton_width_to_check': 120,
    'min_size_threshold_bouton': 30,
    'max_size_threshold_bouton': 400,
    'split_significance_threshold_web': 23,

# Solution 2: Data Input Output 

In [ ]:
"""
Problem: How to abstract data input

Create an API that would abstract where data was coming from

Implementation: Abstract python class 'DataInterfaceDefault'
that defined all the information that users would need to supply
about their volume in order for neurd to run. Define the information
thorugh abstract class methods that need to be overrided

Necessary information (abstract methods): 
1) voxel_to_nm_scaling:
    (default: np.array([1,1,1]))
2) get_align_matrix: 
    a transformation matrix that aligns mesh coordinates
    so apical is point up
    (default: identity matrix)
3) segment_id_to_synapse_df: 
    retriving synapse information for a segment id
    (default implementation: local csv)
4) fetch_segment_id_mesh (not abstract yet): 
    how the meshes are being being retrieved
    (default: local .off file)

Purpose: 
1) if users then subclass DataInterfaceDefault and implement abstract 
functions then pipelines are abstracted

Add on: 
1) If define a parameters_config_filepath list attribute of subclass,
    the vdi parent class will automatically configure all the parameters
2) Can override other properties of vdi that are used in the NEURD pipeline


Conclusion: The vdi subclass is all need to define to adapt NEURD 
for your volume

"""

In [14]:
from neurd.vdi_h01 import volume_data_interface as vdi
vdi.set_parameters_for_directory_modules()

In [15]:
import inspect
import neurd.vdi_h01 as vdi_curr

source_code = inspect.getsource(vdi_curr)
print(source_code)

import numpy as np
from pathlib import Path

from . import vdi_default as vdi_def

parameters_config_filename = "parameters_config_h01.py"
config_filepath = str((
    Path(__file__).parents[0]
    / Path(f"parameter_configs/{parameters_config_filename}")
).absolute())

default_settings = dict(
    source = "h01",
    parameters_config_filepaths = config_filepath,
    synapse_filepath = None,
)

from . import h01_volume_utils as hvu
class DataInterfaceH01(vdi_def.DataInterfaceDefault):
    
    def __init__(
        self,
        **kwargs
        ):
        
        kwargs.update(default_settings)
        super().__init__(
            **kwargs
        )
        
    @property
    def voxel_to_nm_scaling(self):
        return np.array([8,8,33])
    
    def segment_id_to_synapse_df(
        self,
        *args,
        **kwargs):
        return super().segment_id_to_synapse_df(
            *args,
            **kwargs
        )
        
    @property
    def default_low_degree_graph_filte

# Solution 3: Abstracting Storing Data

In [ ]:
"""
Problem: Abstract intermediate/final data products from datajoint

Solution: Store all intermediate products in a python class
(the user can then decide where to output it from there)

Implementation:
datasci_tools/pipeline

"""

In [16]:
from datasci_tools import pipeline

In [20]:
some_nested_dict = dict(
    parameters = dict(
        x = 60,
        y = 70),
    output_somas = 100,
    glia_info = dict(
        glia_1 = 100,
        glia_2=6
    ),
)

stage = pipeline.StageProducts(some_nested_dict)
print(stage)

    glia_info:{'glia_1': 100, 'glia_2': 6}
    output_somas:100
    parameters:{'x': 60, 'y': 70}



In [25]:
P = pipeline.PipelineProducts(
    first_stage = stage,
)

In [27]:
P.stages

['first_stage']